In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pyarrow as pa
import pyarrow.parquet as pq
import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/Optiver/train.csv')
test = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/Optiver/test.csv')
stock0 = pq.read_table('/Users/ericp/OneDrive/Documents/GitHub/Optiver/book_test.parquet/stock_id=0/stock_zero.parquet').to_pandas()
stock0_test = pq.read_table('/Users/ericp/OneDrive/Documents/GitHub/Optiver/trade_test.parquet/stock_id=0/stock_zero.parquet').to_pandas()

In [3]:
order_book_train = glob.glob('/Users/ericp/OneDrive/Documents/GitHub/Optiver/book_train.parquet/*')
trade_book_train = glob.glob('/Users/ericp/OneDrive/Documents/GitHub/Optiver/trade_train.parquet/*')

In [10]:
order_book = []
for book in order_book_train:
    order_book.append(pq.read_table(book).to_pandas())
order_book = pd.concat(order_book, ignore_index = True)

In [11]:
trade_book = []
for book in trade_book_train:
    trade_book.append(pq.read_table(book).to_pandas())
trade_book = pd.concat(trade_book, ignore_index = True)

In [13]:
print(train.shape)
print(test.shape)
print(order_book.shape)
print(trade_book.shape)

(428932, 3)
(3, 3)
(167253289, 10)
(38382741, 5)


In [14]:
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [15]:
print(order_book.head())

   time_id  seconds_in_bucket  bid_price1  ask_price1  bid_price2  ask_price2  \
0        5                  0    1.001422    1.002301     1.00137    1.002353   
1        5                  1    1.001422    1.002301     1.00137    1.002353   
2        5                  5    1.001422    1.002301     1.00137    1.002405   
3        5                  6    1.001422    1.002301     1.00137    1.002405   
4        5                  7    1.001422    1.002301     1.00137    1.002405   

   bid_size1  ask_size1  bid_size2  ask_size2  
0          3        226          2        100  
1          3        100          2        100  
2          3        100          2        100  
3          3        126          2        100  
4          3        126          2        100  


In [16]:
print(trade_book.head())

   time_id  seconds_in_bucket     price  size  order_count
0        5                 21  1.002301   326           12
1        5                 46  1.002778   128            4
2        5                 50  1.002818    55            1
3        5                 57  1.003155   121            5
4        5                 68  1.003646     4            1
